In [1]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

2023-09-07 17:09:27.607008: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 17:09:28.137103: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [3]:
tf.keras.backend.clear_session()

In [4]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [5]:
subject = 'São Paulo - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [6]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_SP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Produção de Cimento (t),São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade,São Paulo - Consumo de Cimento (t)
0,2008,0.809971,1.092396e+09,4.853082e+07,24.143787,9.946828e+08,7770.929330,0.794724,5.483335e+08,2.546644e+09,...,8.071263,18.095190,836.334300,1204.525858,15.181901,11.136027,78.961939,1.773094,0.851565,13476.268000
1,2009,0.812524,1.098669e+09,4.873392e+07,24.141551,9.947096e+08,8316.504885,0.793388,5.302764e+08,2.679945e+09,...,8.100327,17.457437,836.629901,1207.913300,15.173450,11.417327,80.559826,1.537503,0.853009,13304.301000
2,2010,0.814896,1.124010e+09,4.884708e+07,24.146600,9.985217e+08,8835.509117,0.792439,5.046425e+08,2.891930e+09,...,8.064867,17.476576,836.656415,1207.430943,14.880006,11.288684,81.499868,1.367891,0.853860,14534.980000
3,2011,0.817083,1.146637e+09,4.854680e+07,24.134292,1.000762e+09,9192.769768,0.791869,4.854622e+08,3.091988e+09,...,8.053175,17.341412,836.557224,1189.112243,14.815765,11.391381,82.321001,1.335640,0.854127,14871.943000
4,2012,0.818822,1.166721e+09,4.823983e+07,24.089660,1.002188e+09,9435.198835,0.791406,4.867661e+08,3.189506e+09,...,10.058484,16.745399,836.685833,1175.350304,14.895124,11.497643,82.512436,1.341854,0.853708,15159.683000
5,2013,0.821057,1.184231e+09,4.786423e+07,24.041932,1.003187e+09,9197.980881,0.792190,5.225848e+08,3.093872e+09,...,10.842573,15.620656,836.833718,1154.703253,15.020803,11.542975,82.278650,1.326810,0.852933,15089.297000
6,2014,0.822812,1.197826e+09,4.735435e+07,24.001434,1.003012e+09,8264.309556,0.792622,5.676413e+08,2.779340e+09,...,11.415800,15.750987,836.986331,1108.545094,15.247907,11.986366,82.848722,1.532951,0.851595,13714.511286
7,2015,0.823629,1.206909e+09,4.689336e+07,23.951008,1.001922e+09,7069.711323,0.792841,5.179392e+08,2.561731e+09,...,11.865646,15.893940,837.040042,1132.416417,15.520197,12.242333,82.064855,1.539659,0.849644,11767.653429
8,2016,0.832029,1.212819e+09,4.650979e+07,23.922375,1.001305e+09,6155.235871,0.785642,4.524144e+08,2.516859e+09,...,11.874466,15.955560,837.011044,1181.481000,15.455320,11.972800,81.279861,1.362167,0.857989,9811.084286
9,2017,0.831577,1.214213e+09,4.632537e+07,23.910733,1.000929e+09,5693.780428,0.785518,4.213689e+08,2.497710e+09,...,11.486716,16.984321,837.006952,1198.617570,15.304181,12.004953,82.751052,1.502384,0.855534,9918.501000


In [7]:
input_data = data.iloc[:-2, 1:-1]
input_data

,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Produção de Cimento (t),São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade
0,0.809971,1.092396e+09,4.853082e+07,24.143787,9.946828e+08,7770.929330,0.794724,5.483335e+08,2.546644e+09,1.874883e+10,...,0.818381,8.071263,18.095190,836.334300,1204.525858,15.181901,11.136027,78.961939,1.773094,0.851565
1,0.812524,1.098669e+09,4.873392e+07,24.141551,9.947096e+08,8316.504885,0.793388,5.302764e+08,2.679945e+09,2.055015e+10,...,0.819150,8.100327,17.457437,836.629901,1207.913300,15.173450,11.417327,80.559826,1.537503,0.853009
2,0.814896,1.124010e+09,4.884708e+07,24.146600,9.985217e+08,8835.509117,0.792439,5.046425e+08,2.891930e+09,2.342247e+10,...,0.819803,8.064867,17.476576,836.656415,1207.430943,14.880006,11.288684,81.499868,1.367891,0.853860
3,0.817083,1.146637e+09,4.854680e+07,24.134292,1.000762e+09,9192.769768,0.791869,4.854622e+08,3.091988e+09,2.604265e+10,...,0.820344,8.053175,17.341412,836.557224,1189.112243,14.815765,11.391381,82.321001,1.335640,0.854127
4,0.818822,1.166721e+09,4.823983e+07,24.089660,1.002188e+09,9435.198835,0.791406,4.867661e+08,3.189506e+09,2.708753e+10,...,0.820550,10.058484,16.745399,836.685833,1175.350304,14.895124,11.497643,82.512436,1.341854,0.853708
5,0.821057,1.184231e+09,4.786423e+07,24.041932,1.003187e+09,9197.980881,0.792190,5.225848e+08,3.093872e+09,2.523394e+10,...,0.821262,10.842573,15.620656,836.833718,1154.703253,15.020803,11.542975,82.278650,1.326810,0.852933
6,0.822812,1.197826e+09,4.735435e+07,24.001434,1.003012e+09,8264.309556,0.792622,5.676413e+08,2.779340e+09,1.954361e+10,...,0.821534,11.415800,15.750987,836.986331,1108.545094,15.247907,11.986366,82.848722,1.532951,0.851595
7,0.823629,1.206909e+09,4.689336e+07,23.951008,1.001922e+09,7069.711323,0.792841,5.179392e+08,2.561731e+09,1.327596e+10,...,0.821245,11.865646,15.893940,837.040042,1132.416417,15.520197,12.242333,82.064855,1.539659,0.849644
8,0.832029,1.212819e+09,4.650979e+07,23.922375,1.001305e+09,6155.235871,0.785642,4.524144e+08,2.516859e+09,1.011370e+10,...,0.824327,11.874466,15.955560,837.011044,1181.481000,15.455320,11.972800,81.279861,1.362167,0.857989
9,0.831577,1.214213e+09,4.632537e+07,23.910733,1.000929e+09,5693.780428,0.785518,4.213689e+08,2.497710e+09,8.456309e+09,...,0.823356,11.486716,16.984321,837.006952,1198.617570,15.304181,12.004953,82.751052,1.502384,0.855534


In [8]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     13304.301000
1     14534.980000
2     14871.943000
3     15159.683000
4     15089.297000
5     13714.511286
6     11767.653429
7      9811.084286
8      9918.501000
9     10001.077000
10    10458.276000
11    11181.809000
12    12113.838000
13    12383.649000
14             NaN
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [9]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Produção de Cimento (t),São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade
0,-1.601914,-1.819408,1.003478,1.202107,-1.961368,0.116475,1.285446,1.204752,-0.990234,0.166091,...,-1.705828,-1.355043,1.636748,-2.157967,0.548623,0.114751,-1.578154,-2.418314,2.112223,-0.575401
1,-1.272998,-1.675567,1.209427,1.179772,-1.951541,0.498168,0.894167,0.820657,-0.546688,0.428911,...,-1.234652,-1.336507,0.960381,-0.797175,0.643440,0.075193,-0.842756,-1.158674,0.402706,0.032912
2,-0.967315,-1.094542,1.324177,1.230212,-0.558611,0.861272,0.616212,0.275392,0.158672,0.847995,...,-0.834215,-1.359122,0.980679,-0.675119,0.629939,-1.298380,-1.179065,-0.417623,-0.828049,0.391069
3,-0.685507,-0.575726,1.019683,1.107266,0.259840,1.111217,0.449046,-0.132596,0.824346,1.230291,...,-0.503056,-1.366579,0.837331,-1.131742,0.117185,-1.599084,-0.910586,0.229689,-1.062071,0.503752
4,-0.461531,-0.115211,0.708407,0.661426,0.781023,1.280824,0.313381,-0.104860,1.148829,1.382743,...,-0.376604,-0.087675,0.205232,-0.539690,-0.268021,-1.227616,-0.632790,0.380601,-1.016982,0.327130
5,-0.173555,0.286253,0.327534,0.184658,1.145924,1.114863,0.543033,0.657046,0.830615,1.112297,...,0.060098,0.412385,-0.987611,0.141096,-0.845947,-0.639328,-0.514278,0.196304,-1.126146,0.001028
6,0.052526,0.597984,-0.189503,-0.219884,1.082248,0.461651,0.669744,1.615453,-0.215960,0.282052,...,0.226730,0.777966,-0.849388,0.843646,-2.137947,0.423719,0.644870,0.645700,0.369675,-0.562814
7,0.157871,0.806234,-0.656965,-0.723606,0.683738,-0.374108,0.733766,0.558230,-0.940033,-0.632425,...,0.049709,1.064859,-0.697781,1.090901,-1.469772,1.698271,1.314042,0.027766,0.418348,-1.384206
8,1.240079,0.941752,-1.045917,-1.009627,0.458458,-1.013890,-1.375468,-0.835563,-1.089341,-1.093812,...,1.938197,1.070484,-0.632429,0.957410,-0.096419,1.394591,0.609405,-0.591059,-0.869586,2.130112
9,1.181857,0.973717,-1.232918,-1.125922,0.321024,-1.336731,-1.411648,-1.495939,-1.153056,-1.335633,...,1.343468,0.823193,0.458620,0.938576,0.383246,0.687130,0.693462,0.568705,0.147869,1.096015


In [10]:
reshaped_train_input = dfToInputRNN(train_input)

In [11]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 1.19528157,  0.74815575, -1.12518296, -1.11567726,
         -0.20940454, -1.20055538, -1.24517775, -1.0453709 ,
          1.21859064, -0.99976018,  0.79428371,  1.15575749,
         -0.52559858,  1.30260559, -0.55253046, -0.25130359,
          1.38787176,  1.47154451, -1.80649616,  1.09296291,
          0.94281138,  0.52836084,  0.61258103,  0.87656257,
          1.47801536,  1.71982086,  1.28221181,  1.39086472,
          1.58209844,  1.08872091,  0.55212962, -1.81398118,
          1.44113674,  1.11191094,  1.30470835,  0.86390516,
         -1.67365968, -1.70738939, -1.55813196, -0.48498301,
         -0.75714639,  0.40770893, -0.27394734,  2.02323219,
          1.2908951 ,  1.70465513,  1.81148084,  1.39968653,
         -1.36191442,  1.28024593,  1.11054795,  0.12662799,
          1.40414112,  1.40468999,  1.45643959,  1.58509336,
          1.32283157,  1.30222047,  1.55138602,  1.91801602,
         -0.77489784,  2.32098884, -1.1942541 , -1.17081042,
          0.20821245,  0

In [12]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     13304.301000
1     14534.980000
2     14871.943000
3     15159.683000
4     15089.297000
5     13714.511286
6     11767.653429
7      9811.084286
8      9918.501000
9     10001.077000
10    10458.276000
11    11181.809000
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [13]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    11181.809
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [14]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Produção de Cimento (t),São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade
12,1.001135,0.409525,-0.926633,-1.322522,-1.033191,-0.97248,-1.385738,1.108405,2.635823,-0.189959,...,-1.092902,0.514165,-0.901047,-0.751547,0.232288,-0.363994,-0.503159,-0.998268,-1.560012,-2.468148


In [15]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 1.00113455,  0.40952509, -0.9266328 , -1.32252187,
         -1.03319098, -0.97248018, -1.38573782,  1.10840491,
          2.63582309, -0.18995911,  0.72640489,  2.66833308,
         -0.032632  ,  0.98802435,  0.24980494, -0.08955808,
          1.85434394,  2.02706568, -2.21150703,  1.42938591,
          0.85132668,  0.14092386,  0.30902324,  1.00267904,
          2.27970137,  1.11272593,  0.61007902,  0.89064538,
          1.42231438,  1.02650562,  1.47127593, -1.53201473,
          1.73503858,  1.66610613,  1.0456894 ,  0.55289259,
         -1.81216593, -2.10342296, -2.55377478,  0.41940628,
         -1.75756445,  0.241023  , -0.98659345,  1.52184916,
          1.01398607,  1.49290967,  1.40764092,  1.87930411,
         -2.51892072,  2.83356214,  1.58620924, -0.02524128,
          1.54231542,  1.5176119 ,  1.7355269 ,  1.87941982,
          1.54198129,  1.51276712,  1.83795594,  2.62210693,
          1.04993174,  0.96476493, -1.48277861, -2.26044738,
         -1.09290187,  0

In [16]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    12113.838
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [20]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [21]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [22]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[845644770, 2909323468, 774016587, 3520921306, 3729471184]


Step: 0 ___________________________________________
loss: 71661.6015625
winner_seed: 845644770


Step: 1 ___________________________________________
loss: 207339.421875


Step: 2 ___________________________________________
loss: 121887.5625


Step: 3 ___________________________________________
loss: 94210.75


Step: 4 ___________________________________________
loss: 167481.03125


final_seed: 845644770


In [23]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 3s 3s/step - loss: 160215216.0000
Epoch 2/10000
1/1 [==============================] - 0s 5ms/step - loss: 153458640.0000
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 34407836.0000
Epoch 4/10000
1/1 [==============================] - 0s 4ms/step - loss: 2838060.0000
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 7656481.5000
Epoch 6/10000
1/1 [==============================] - 0s 5ms/step - loss: 1924915.1250
Epoch 7/10000
1/1 [==============================] - 0s 5ms/step - loss: 1038484.1875
Epoch 8/10000
1/1 [==============================] - 0s 5ms/step - loss: 1328922.2500
Epoch 9/10000
1/1 [==============================] - 0s 5ms/step - loss: 1744534.6250
Epoch 10/10000
1/1 [==============================] - 0s 5ms/step - loss: 1021764.0625
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 1421826.0000
Epoch 12/10000
1/1 [============================

1/1 [==============================] - 0s 5ms/step - loss: 906144.6875
Epoch 96/10000
1/1 [==============================] - 0s 5ms/step - loss: 1232510.8750
Epoch 97/10000
1/1 [==============================] - 0s 5ms/step - loss: 1403597.3750
Epoch 98/10000
1/1 [==============================] - 0s 4ms/step - loss: 1566842.0000
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 1572183.1250
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 1526657.0000
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 1979146.1250
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 1319974.8750
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 721299.7500
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 802756.9375
Epoch 105/10000
1/1 [==============================] - 0s 5ms/step - loss: 928207.0000
Epoch 106/10000
1/1 [==============================] - 0

1/1 [==============================] - 0s 4ms/step - loss: 1675531.8750
Epoch 189/10000
1/1 [==============================] - 0s 5ms/step - loss: 775906.3125
Epoch 190/10000
1/1 [==============================] - 0s 4ms/step - loss: 1025344.5000
Epoch 191/10000
1/1 [==============================] - 0s 4ms/step - loss: 769773.8125
Epoch 192/10000
1/1 [==============================] - 0s 5ms/step - loss: 1177399.3750
Epoch 193/10000
1/1 [==============================] - 0s 5ms/step - loss: 596401.1875
Epoch 194/10000
1/1 [==============================] - 0s 4ms/step - loss: 882613.6875
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 735993.8125
Epoch 196/10000
1/1 [==============================] - 0s 5ms/step - loss: 1779756.3750
Epoch 197/10000
1/1 [==============================] - 0s 5ms/step - loss: 1600164.6250
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 1853939.3750
Epoch 199/10000
1/1 [==============================] 

1/1 [==============================] - 0s 4ms/step - loss: 1288822.8750
Epoch 282/10000
1/1 [==============================] - 0s 4ms/step - loss: 1736094.3750
Epoch 283/10000
1/1 [==============================] - 0s 4ms/step - loss: 726961.8125
Epoch 284/10000
1/1 [==============================] - 0s 5ms/step - loss: 1178400.3750
Epoch 285/10000
1/1 [==============================] - 0s 4ms/step - loss: 972968.7500
Epoch 286/10000
1/1 [==============================] - 0s 5ms/step - loss: 852311.8125
Epoch 287/10000
1/1 [==============================] - 0s 4ms/step - loss: 853949.5000
Epoch 288/10000
1/1 [==============================] - 0s 4ms/step - loss: 441565.5000
Epoch 289/10000
1/1 [==============================] - 0s 5ms/step - loss: 1180449.2500
Epoch 290/10000
1/1 [==============================] - 0s 4ms/step - loss: 1098921.3750
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 1192359.8750
Epoch 292/10000
1/1 [==============================] 

Epoch 375/10000
1/1 [==============================] - 0s 4ms/step - loss: 610598.8125
Epoch 376/10000
1/1 [==============================] - 0s 5ms/step - loss: 504367.7188
Epoch 377/10000
1/1 [==============================] - 0s 5ms/step - loss: 1180794.3750
Epoch 378/10000
1/1 [==============================] - 0s 4ms/step - loss: 815446.9375
Epoch 379/10000
1/1 [==============================] - 0s 4ms/step - loss: 1049119.3750
Epoch 380/10000
1/1 [==============================] - 0s 4ms/step - loss: 733617.5625
Epoch 381/10000
1/1 [==============================] - 0s 4ms/step - loss: 860602.1875
Epoch 382/10000
1/1 [==============================] - 0s 4ms/step - loss: 774936.0000
Epoch 383/10000
1/1 [==============================] - 0s 4ms/step - loss: 466226.8750
Epoch 384/10000
1/1 [==============================] - 0s 4ms/step - loss: 596731.2500
Epoch 385/10000
1/1 [==============================] - 0s 5ms/step - loss: 368710.7188
Epoch 386/10000
1/1 [====================

...

1/1 [==============================] - 0s 4ms/step - loss: 536956.6250
Epoch 1775/10000
1/1 [==============================] - 0s 5ms/step - loss: 656031.8125
Epoch 1776/10000
1/1 [==============================] - 0s 4ms/step - loss: 792816.5000
Epoch 1777/10000
1/1 [==============================] - 0s 4ms/step - loss: 635418.4375
Epoch 1778/10000
1/1 [==============================] - 0s 4ms/step - loss: 1127159.6250
Epoch 1779/10000
1/1 [==============================] - 0s 4ms/step - loss: 322411.2500
Epoch 1780/10000
1/1 [==============================] - 0s 4ms/step - loss: 978608.6875
Epoch 1781/10000
1/1 [==============================] - 0s 4ms/step - loss: 532362.5625
Epoch 1782/10000
1/1 [==============================] - 0s 4ms/step - loss: 486606.5000
Epoch 1783/10000
1/1 [==============================] - 0s 4ms/step - loss: 782858.6875
Epoch 1784/10000
1/1 [==============================] - 0s 4ms/step - loss: 613456.0000
Epoch 1785/10000
1/1 [==========================

1/1 [==============================] - 0s 5ms/step - loss: 453122.5312
Epoch 1868/10000
1/1 [==============================] - 0s 4ms/step - loss: 449870.5312
Epoch 1869/10000
1/1 [==============================] - 0s 4ms/step - loss: 265652.3125
Epoch 1870/10000
1/1 [==============================] - 0s 4ms/step - loss: 515913.0938
Epoch 1871/10000
1/1 [==============================] - 0s 4ms/step - loss: 1860226.6250
Epoch 1872/10000
1/1 [==============================] - 0s 4ms/step - loss: 1167747.5000
Epoch 1873/10000
1/1 [==============================] - 0s 5ms/step - loss: 582620.3750
Epoch 1874/10000
1/1 [==============================] - 0s 4ms/step - loss: 692209.3125
Epoch 1875/10000
1/1 [==============================] - 0s 4ms/step - loss: 1217788.7500
Epoch 1876/10000
1/1 [==============================] - 0s 4ms/step - loss: 232961.8594
Epoch 1877/10000
1/1 [==============================] - 0s 4ms/step - loss: 479560.9062
Epoch 1878/10000
1/1 [========================

1/1 [==============================] - 0s 4ms/step - loss: 473264.5938
Epoch 1961/10000
1/1 [==============================] - 0s 4ms/step - loss: 586909.1875
Epoch 1962/10000
1/1 [==============================] - 0s 4ms/step - loss: 1008954.1875
Epoch 1963/10000
1/1 [==============================] - 0s 5ms/step - loss: 456321.4688
Epoch 1964/10000
1/1 [==============================] - 0s 4ms/step - loss: 669014.4375
Epoch 1965/10000
1/1 [==============================] - 0s 5ms/step - loss: 680738.0625
Epoch 1966/10000
1/1 [==============================] - 0s 4ms/step - loss: 789923.8125
Epoch 1967/10000
1/1 [==============================] - 0s 4ms/step - loss: 586504.8750
Epoch 1968/10000
1/1 [==============================] - 0s 4ms/step - loss: 819951.5000
Epoch 1969/10000
1/1 [==============================] - 0s 5ms/step - loss: 601866.1250
Epoch 1970/10000
1/1 [==============================] - 0s 4ms/step - loss: 302151.3125
Epoch 1971/10000
1/1 [==========================

1/1 [==============================] - 0s 4ms/step - loss: 1136340.0000
Epoch 2054/10000
1/1 [==============================] - 0s 4ms/step - loss: 628801.0000
Epoch 2055/10000
1/1 [==============================] - 0s 5ms/step - loss: 759295.8125
Epoch 2056/10000
1/1 [==============================] - 0s 4ms/step - loss: 1316056.1250
Epoch 2057/10000
1/1 [==============================] - 0s 4ms/step - loss: 779704.5625
Epoch 2058/10000
1/1 [==============================] - 0s 4ms/step - loss: 822859.6875
Epoch 2059/10000
1/1 [==============================] - 0s 5ms/step - loss: 292168.2812
Epoch 2060/10000
1/1 [==============================] - 0s 4ms/step - loss: 266037.0938
Epoch 2061/10000
1/1 [==============================] - 0s 4ms/step - loss: 1182001.7500
Epoch 2062/10000
1/1 [==============================] - 0s 4ms/step - loss: 538250.0625
Epoch 2063/10000
1/1 [==============================] - 0s 4ms/step - loss: 463734.6562
Epoch 2064/10000
1/1 [========================

1/1 [==============================] - 0s 5ms/step - loss: 777145.1875
Epoch 2147/10000
1/1 [==============================] - 0s 4ms/step - loss: 757354.8125
Epoch 2148/10000
1/1 [==============================] - 0s 5ms/step - loss: 924044.3125
Epoch 2149/10000
1/1 [==============================] - 0s 4ms/step - loss: 410014.0938
Epoch 2150/10000
1/1 [==============================] - 0s 4ms/step - loss: 740138.3125
Epoch 2151/10000
1/1 [==============================] - 0s 4ms/step - loss: 1507365.5000
Epoch 2152/10000
1/1 [==============================] - 0s 4ms/step - loss: 319798.5625
Epoch 2153/10000
1/1 [==============================] - 0s 4ms/step - loss: 727158.8125
Epoch 2154/10000
1/1 [==============================] - 0s 5ms/step - loss: 736883.4375
Epoch 2155/10000
1/1 [==============================] - 0s 4ms/step - loss: 575789.2500
Epoch 2156/10000
1/1 [==============================] - 0s 4ms/step - loss: 553072.6250
Epoch 2157/10000
1/1 [==========================

In [26]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 15ms/step
[12113.838](test_target) - [10485.844](prediction) = 1627.9942499999997


In [27]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.13439128457884278

In [30]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [31]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Produção de Cimento (t),São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade
0,-1.239414,-0.924099,-1.323126,-0.093200,-0.714550,-1.234806,1.287736,1.148205,-1.122043,-1.121418,...,-1.256499,-0.489673,1.413722,-1.410322,-1.401911,0.736997,-1.258070,-1.315911,1.285551,-1.315803
1,0.029885,-0.465059,0.229123,-1.175483,-0.699637,0.020377,-0.137605,0.140893,-0.184474,-0.185492,...,0.066193,1.393821,-0.739164,0.614359,0.862152,0.676789,1.188450,0.209310,-0.132366,0.209017
2,1.209529,1.389158,1.094002,1.268682,1.414187,1.214429,-1.150131,-1.289098,1.306518,1.306910,...,1.190306,-0.904148,-0.674558,0.795962,0.539759,-1.413786,0.069619,1.106602,-1.153185,1.106786


train_target:


,São Paulo - Consumo de Cimento (t)
0,13304.301
1,14534.980
2,14871.943


test_input:


...

,São Paulo - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Produção de Cimento (t),São Paulo - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,São Paulo - IDH,São Paulo - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),São Paulo - IDH Longevidade
12,1.001135,0.409525,-0.926633,-1.322522,-1.033191,-0.97248,-1.385738,1.108405,2.635823,-0.189959,...,-1.092902,0.514165,-0.901047,-0.751547,0.232288,-0.363994,-0.503159,-0.998268,-1.560012,-2.468148


test_target:


,São Paulo - Consumo de Cimento (t)
12,12113.838


1/1 [==============================] - 1s 605ms/step
Error: 1627.9942499999997




[14459.2275390625,
 14465.208984375,
 10569.1396484375,
 11368.8984375,
 10436.9609375,
 10673.421875,
 9867.751953125,
 10198.357421875,
 10089.2685546875,
 10485.84375]

In [32]:
display(targets)
display(predictions)

[15159.683,
 15089.297,
 13714.511285714289,
 11767.65342857143,
 9811.084285714289,
 9918.501,
 10001.077,
 10458.276,
 11181.809,
 12113.838]

[14459.2275390625,
 14465.208984375,
 10569.1396484375,
 11368.8984375,
 10436.9609375,
 10673.421875,
 9867.751953125,
 10198.357421875,
 10089.2685546875,
 10485.84375]

In [33]:
mae = mean_absolute_error(predictions, targets)
mae

936.3245952008929

In [34]:
porcentage = mae/np.mean(targets)
porcentage

0.07854035664596382